<a href="https://colab.research.google.com/github/ValliamaiRM/Sunspot_MagneticField_WFA/blob/main/MagneticField_Sunspot_WFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing all the neccesary libriries
%matplotlib tk
import astropy.io.fits
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
from sklearn.linear_model import LinearRegression
import math

In [ ]:
#locating the data fits file 
base_path = r'G:\Sunspot Data'
filename_list = list()
filename_list.append("\Stokeim_20160428_12035179.fits")
filename_list.append("\Stokeim_20160428_12043084.fits")
filename_list.append("\Stokeim_20160428_12051015.fits")
filename_list.append("\Stokeim_20160428_12054932.fits")
filename_list.append("\Stokeim_20160428_12062853.fits")
filename_list.append("\Stokeim_20160428_12070789.fits")
filename_list.append("\Stokeim_20160428_12074717.fits")
filename_list.append("\Stokeim_20160428_12082617.fits")
filename_list.append("\Stokeim_20160428_12090537.fits")
filename_list.append("\Stokeim_20160428_12094484.fits")
filename_list.append("\Stokeim_20160428_12102409.fits")
filename_list.append("\Stokeim_20160428_12110343.fits")
filename_list.append("\Stokeim_20160428_12114285.fits")
filename_list.append("\Stokeim_20160428_12122215.fits")
print(filename_list[0])

\Stokeim_20160428_12035179.fits


In [ ]:
#Functions for line of sight magnetic field
#The following function computes the numerator value of the los magnetic field as given in Centeno(2018)
def numerator_for_LOS_magnetic_field(l,ypixelno,SI,SV):
    x = np.array(wl[l-1:l+1]).reshape((-1, 1))
    y = np.array(SI[ypixelno,l-1:l+1])
    model = LinearRegression().fit(x, y)
    slope=model.coef_
    slopebyI=slope/SI[ypixelno,l]
    num=slopebyI*SV[ypixelno,l]
    return num
#The following function computes the denominator value of the los magnetic field as given in Centeno(2018)
def denominator_for_LOS_magnetic_field(l,pixelno,SI):
    x = np.array(wl[l-1:l+1]).reshape((-1, 1))
    y = np.array(SI[ypixelno,l-1:l+1])
    model = LinearRegression().fit(x, y)
    slope=model.coef_
    slopebyI=slope/SI[ypixelno,l]
    den= slopebyI**2
    return den
#The following function computes the los magnetic field for a given y pixel no.
def PhotosphericBlosForGivenYPixel(l,SI,SV,ypixelno):
    wfe=35442.63616
    numerator=0
    denominator=0
    for i in range(len(l)):
        numerator+=numerator_for_LOS_magnetic_field(l[i],ypixelno,SI,SV)
        denominator+=denominator_for_LOS_magnetic_field(l[i],ypixelno,SI)
    Blosvalue= (-1)*(wfe*numerator)/denominator
    return Blosvalue
def chromosphericBlosForGivenYPixel(l,SI,SV,ypixelno):
    w_halpha=46034.10475
    numerator=0
    denominator=0
    for i in range(len(l)):
        numerator+=numerator_for_LOS_magnetic_field(l[i],ypixelno,SI,SV)
        denominator+=denominator_for_LOS_magnetic_field(l[i],ypixelno,SI)
    Blosvalue= (-1)*(w_halpha*numerator)/denominator
    return Blosvalue
#The following function computes the numerator value of the Transverse magnetic field as given in Centeno(2018)
def numerator_for_transverse_magnetic_field(l,ypixelno,SI,SU,SQ,wl,ct):
    x = np.array(wl[l-1:l+1]).reshape((-1, 1))
    y = np.array(SI[ypixelno,l-1:l+1])
    model = LinearRegression().fit(x, y)
    slope=model.coef_
    slopebyI=abs(slope/SI[ypixelno,l])
    SL=((SQ[ypixelno,l])**2)+(((SU[ypixelno,l]))**2)
    num=(4/3)*(1/ct)*slopebyI*(SL**0.5)*abs(((wl[l]-6569.231)))
    return num
#The following function computes the denominater value of the transverse magnetic field as given in Centeno(2018)
def denominator_for_transverse_magnetic_field(l,pixelno,SI,wl):
    x = np.array(wl[l-1:l+1]).reshape((-1, 1))
    y = np.array(SI[ypixelno,l-1:l+1])
    model = LinearRegression().fit(x, y)
    slope=model.coef_
    slopebyI=abs(slope/SI[ypixelno,l])
    den=((slopebyI**2))*(abs(((wl[l]-6569.231))))**2
    return den
#The following function computes the transverse magnetic field for a given y pixel no.
def PhotosphericBTransForGivenYPixel(l,SI,SU,SQ,ypixelno,wl):
    CTfe=5.682762555e-4
    numerator=0
    denominator=0
    for i in range(len(l)):
        numerator+=numerator_for_transverse_magnetic_field(l[i],ypixelno,SI,SU,SQ,wl,CTfe)
        denominator+=denominator_for_transverse_magnetic_field(l[i],ypixelno,SI,wl)
    Blosvalue= (((numerator))/(denominator))**0.5
    return Blosvalue
def chrompsphericBTransForGivenYPixel(l,SI,SU,SQ,ypixelno):
    CT_halpha=4.366730722e-4
    numerator=0
    denominator=0
    for i in range(len(l)):
        numerator+=numerator_for_transverse_magnetic_field(l[i],ypixelno,SI,SU,SQ,wl,CT_halpha)
        denominator+=denominator_for_transverse_magnetic_field(l[i],ypixelno,SI,wl)
    Blosvalue= (((4/3)*(1/CT_halpha)*(numerator))/(denominator))**0.5
    return Blosvalue

In [ ]:
#Pixel to wavelength conversion
wl=[]
for i in range(990):
    wl.append((i+1)*0.011298+6559.3)

In [ ]:
#Creating the sunspot continuum image using wavelengths in pixel no. 700 to 750
ssimage=np.zeros(shape=(277,14))
SS=np.ndarray(shape=(1,277,14))
def continuumIntensity(SI,pixelno):
    continuumIntensity= np.average(SI[pixelno,700:750])
    return continuumIntensity
for filename in range(14):
    data=astropy.io.fits.getdata(base_path+filename_list[filename])
    SI=data[0,]
    SI=SI[:,::-1]
    SV=data[3,]
    SV=SV[:,::-1]
    for pixelno in range(277):
        ssimage[pixelno,filename]=(continuumIntensity(SI,pixelno))
        SS[0]=ssimage



plt.imshow(SS[0],aspect=(1/7),origin=0)


In [ ]:
#Creating los magnetic field image
#The following l values are the index of the wavelengths close to the Fe 1 line 
l=[873,874,875,882,883,884]
blos_photospheric=np.zeros(shape=(277,14))
BLOS_Photospheric=np.ndarray(shape=(1,277,14))
for filename in range(14):
    data=astropy.io.fits.getdata(base_path+filename_list[filename])
    SI=data[0,]
    SI=SI[:,::-1]
    SV=data[3,]
    SV=SV[:,::-1]
    for ypixelno in range(277):
        blos_photospheric[ypixelno,filename]=(PhotosphericBlosForGivenYPixel(l,SI,SV,ypixelno))
BLOS_Photospheric[0]=blos_photospheric
plt.imshow(BLOS_Photospheric[0],aspect=(1/7),origin=0,cmap='gray')

In [ ]:
#Creating transverse magnetic field image 
btrans_photospheric=np.zeros(shape=(277,14))
BTRANS_Photospheric=np.ndarray(shape=(1,277,14))
l=[878]
for filename in range(14):
    data=astropy.io.fits.getdata(base_path+filename_list[filename])
    SI=data[0,]
    SI=SI[:,::-1]
    SQ=data[1,]
    SQ=SQ[:,::-1]
    SU=data[2,]
    SU=SU[:,::-1]
    for ypixelno in range(277):
        btrans_photospheric[ypixelno,filename]=(PhotosphericBTransForGivenYPixel(l,SI,SU,SQ,ypixelno,wl))
BTRANS_Photospheric[0]=btrans_photospheric
plt.imshow(BTRANS_Photospheric[0],aspect=(1/7),origin=0)

In [ ]:
#Creating Choromospheric los magnetic field image
#The following lt values are the index of the wavelengths close to the H alpha line(6560.45-6560.49 & 6560.59-6560.63 A) 
lt=[290,291,292,293,294,295,320,321,322]
blos_chromospheric=np.zeros(shape=(277,14))
BLOS_chromospheric=np.ndarray(shape=(1,277,14))
for filename in range(14):
    data=astropy.io.fits.getdata(base_path+filename_list[filename])
    SI=data[0,]
    SI=SI[:,::-1]
    SV=data[3,]
    SV=SV[:,::-1]
    for ypixelno in range(277):
        blos_chromospheric[ypixelno,filename]=(chromosphericBlosForGivenYPixel(lt,SI,SV,ypixelno))
BLOS_chromospheric[0]=blos_chromospheric
plt.imshow(BLOS_chromospheric[0],aspect=(1/7),origin=0)

In [ ]:
#Creating transverse magnetic field image 
btrans_chromospheric=np.zeros(shape=(277,14))
BTRANS_chromospheric=np.ndarray(shape=(1,277,14))
lt=[308]
for filename in range(14):
    data=astropy.io.fits.getdata(base_path+filename_list[filename])
    SI=data[0,]
    SI=SI[:,::-1]
    SQ=data[1,]
    SQ=SQ[:,::-1]
    SU=data[2,]
    SU=SU[:,::-1]
    for ypixelno in range(277):
        btrans_chromospheric[ypixelno,filename]=(chrompsphericBTransForGivenYPixel(lt,SI,SU,SQ,ypixelno))
BTRANS_chromospheric[0]=btrans_chromospheric
plt.imshow(BTRANS_chromospheric[0],aspect=(1/7),origin=0)


In [ ]:
#plotting contours of same intensity at intensity values - 6.2e4,6.4e4,6.8e4,7e4,7.2e4)
from skimage import measure
fig,ax=plt.subplots()
ax.imshow(SS[0],aspect=(1/7),origin=0)
contours0=measure .find_contours(SS[0],6.28e4)
for n, contour in enumerate(contours0):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours1=measure .find_contours(SS[0],6.3e4)
for n, contour in enumerate(contours1):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours2=measure .find_contours(SS[0],6.34e4)
for n, contour in enumerate(contours2):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours3=measure .find_contours(SS[0],6.4e4)
for n, contour in enumerate(contours3):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours4=measure .find_contours(SS[0],6.5e4)
for n, contour in enumerate(contours4):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours5=measure .find_contours(SS[0],6.6e4)
for n, contour in enumerate(contours5):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours6=measure .find_contours(SS[0],6.8e4)
for n, contour in enumerate(contours6):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours7=measure .find_contours(SS[0],7e4)
for n, contour in enumerate(contours7):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours8=measure .find_contours(SS[0],7.2e4)
for n, contour in enumerate(contours8):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)
contours9=measure .find_contours(SS[0],7.4e4)
for n, contour in enumerate(contours9):
    ax.plot(contour[:,1],contour[:,0],linewidth=0.5)



ax.set_xticks([])
ax.set_yticks([])

plt.show()


In [ ]:
#A function to find the mean/average magnetic field over a given contour passed as argument
def Average_B_ofGivenContour(contour,B):
    Bi=0
    averageB=0
    for i in range(len(contour[0])):
        y=int(contour[0][i][0])
        x=int(contour[0][i][1])
        Bi+=B[0,y,x]
    averageB=Bi/(len(contour[0]))
    return averageB

In [ ]:
#Creating an array 'AverageBlosPhotospheric' to store the mean Blos over the different contours.
AverageBlosPhotospheric=[]
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours0,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours1,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours2,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours3,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours4,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours5,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours6,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours7,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours8,BLOS_Photospheric))
AverageBlosPhotospheric.append(Average_B_ofGivenContour(contours9,BLOS_Photospheric))
print(AverageBlosPhotospheric)

[904.3580933108533, 900.7653909080419, 884.5746107645518, 844.8773646980181, 808.2893852051109, 774.7256089882101, 668.6373722281934, 597.6315538576621, 432.1661141063224, 347.4327274047527]


In [ ]:
#Creating an array 'AverageBtransPhotospheric' to store the mean BTrans over the different contours.
AverageBtransPhotospheric=[]
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours0,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours1,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours2,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours3,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours4,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours5,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours6,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours7,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours8,BTRANS_Photospheric))
AverageBtransPhotospheric.append(Average_B_ofGivenContour(contours9,BTRANS_Photospheric))
print(AverageBtransPhotospheric)

[771.7440358862382, 809.3994952685858, 767.1143769785551, 765.625986241543, 784.654180752413, 817.5202974730851, 821.4118269834707, 792.211439590386, 758.1215696694343, 675.1213477525538]


In [ ]:
#Creating an array 'AverageBloschromospheric' to store the mean Blos over the different contours.
AverageBloschromospheric=[]
AverageBloschromospheric.append(Average_B_ofGivenContour(contours0,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours1,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours2,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours3,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours4,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours5,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours6,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours7,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours8,BLOS_chromospheric))
AverageBloschromospheric.append(Average_B_ofGivenContour(contours9,BLOS_chromospheric))
print(AverageBloschromospheric)

[750.7155821098648, 751.8502136241045, 741.5420357496927, 704.8022280466566, 696.6283263775736, 691.8665678087971, 649.5419982215996, 600.1930586002757, 482.6294393151947, 421.5478929925497]


In [ ]:
#Computing the total magnetic field strength in the photospheric height 
Total_B_Photospheric=[]
for i in range(9):
    Bx=AverageBlosPhotospheric[i]**2
    By=AverageBtransPhotospheric[i]**2
    Total_B_Photospheric.append((Bx+By)**0.5)
print(Total_B_Photospheric)

[1188.8870500862652, 1210.9937375555496, 1170.8700651124595, 1140.1758251197066, 1126.5051769110946, 1126.2944579472255, 1059.1474519860521, 992.3519734369431, 872.648763581205]


In [ ]:
for k in range(int(len(contours0[0]))):
    x=[6]
    y=[118]
    if contours0[0][k][1]>6:
        angle=contours0[0][k][0]/contours0[0][k][1]

        for a in range(int(len(contours1[0]))):
            if contours1[0][a][1]>6:
                anglenow=contours1[0][a][0]/contours1[0][a][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours1[0][a][0] not in y and contours1[0][a][1] not in x:
                        y.append(contours1[0][a][0])
                        x.append(contours1[0][a][1])
                        break

        for b in range(int(len(contours2[0]))):
            if contours2[0][b][1]>6:
                anglenow=contours2[0][b][0]/contours2[0][b][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours2[0][b][0] not in y and contours2[0][b][1] not in x:
                        y.append(contours2[0][b][0])
                        x.append(contours2[0][b][1])
                        break

        for c in range(int(len(contours3[0]))):
            if contours3[0][c][1]>6:
                anglenow=contours3[0][c][0]/contours3[0][c][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours3[0][c][0] not in y and contours3[0][c][1] not in x:
                        y.append(contours3[0][c][0])
                        x.append(contours3[0][c][1])
                        break

        for d in range(int(len(contours4[0]))):
            if contours4[0][d][1]>6:
                anglenow=contours4[0][d][0]/contours4[0][d][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours4[0][d][0] not in y and contours4[0][d][1] not in x:
                        y.append(contours4[0][d][0])
                        x.append(contours4[0][d][1])
                        break
        for e in range(int(len(contours5[0]))):
            if contours5[0][e][1]>6:
                anglenow=contours5[0][e][0]/contours5[0][e][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours5[0][e][0] not in y and contours5[0][e][1] not in x:
                        y.append(contours5[0][e][0])
                        x.append(contours5[0][e][1])
                        break
        for f in range(int(len(contours6[0]))):
            if contours6[0][f][1]>6:
                anglenow=contours6[0][f][0]/contours6[0][f][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours6[0][f][0] not in y and contours6[0][f][1] not in x:
                        y.append(contours6[0][f][0])
                        x.append(contours6[0][f][1])
                        break
        for g in range(int(len(contours7[0]))):
            if contours7[0][g][1]>6:
                anglenow=contours7[0][g][0]/contours7[0][g][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours7[0][g][0] not in y and contours7[0][g][1] not in x:
                        y.append(contours7[0][g][0])
                        x.append(contours7[0][g][1])
                        break
        for h in range(int(len(contours8[0]))):
            if contours8[0][h][1]>6:
                anglenow=contours8[0][h][0]/contours8[0][h][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours8[0][h][0] not in y and contours8[0][h][1] not in x:
                        y.append(contours8[0][h][0])
                        x.append(contours8[0][h][1])
                        break
        for i in range(int(len(contours9[0]))):
            if contours9[0][i][1]>6:
                anglenow=contours9[0][i][0]/contours9[0][i][1]
                if np.isclose(anglenow,angle,0.011):
                    if contours9[0][i][0] not in y and contours9[0][i][1] not in x:
                        y.append(contours9[0][i][0])
                        x.append(contours9[0][i][1])
                        break
plt.plot(x,y)
contoursXvalue1=x
contoursYvalue1=y
print(x)
print(y)


[6, 6.045646065412429, 6.1165072183306535, 6.185433709115081, 6.338753219793457, 6.435369140288579, 6.6123083088219925, 6.81604988537047, 7.065410281317098, 7.3350836959746815]
[118, 120.0, 121.0, 123.0, 125.0, 128.0, 132.0, 136.0, 139.0, 145.0]


In [ ]:
x=[6.045646065412429, 6.1165072183306535, 6.185433709115081, 6.338753219793457, 6.435369140288579, 6.6123083088219925, 6.81604988537047, 7.065410281317098, 7.3350836959746815]
y=[120.0, 121.0, 123.0, 125.0, 128.0, 132.0, 136.0, 139.0, 145.0]
radius=[]
for i in range(8):
    r=(y[i]-118)**2+(x[i]-6)**2
    radius.append(r**0.5)
print(radius)
normalisedRadius=[]
for j in range(8):
    normalisedRadius.append(radius[j]/radius[7])
print(normalisedRadius)

[2.000520823007758, 3.002261469613056, 5.003437384486407, 7.008191902617995, 10.009472827692555, 14.01338365510102, 18.01848876613722, 21.02700879981592]
[0.09514053292379233, 0.1427811962317407, 0.23795288393707284, 0.33329476243332085, 0.47602932604375864, 0.6664468440810135, 0.8569211597179225, 1.0]


In [ ]:
plt.plot(normalisedRadius[1:8],AverageBlosPhotospheric[2:9])
plt.plot(normalisedRadius[1:8],AverageBtransPhotospheric[2:9])

In [ ]:
plt.plot(normalisedRadius[1:8],AverageBloschromospheric[2:9])

In [ ]:
plt.plot(normalisedRadius[1:8],Total_B_Photospheric[2:9])
plt.plot(normalisedRadius[1:8],AverageBloschromospheric[2:9])